# 06_231226_PRSadvance

******************************

## 使用するデータのチェック

In [12]:
%%bash
#PRSを計算した時にQCしたトンガサンプルのチェック
ls ~/hmiwa1/storage/hmiwa-lab/05_231211_RePRSTrials/targetdata/Tongan.QC.FIXP.phased.annotated.vcf.gz

/home/hmiwa/hmiwa1/storage/hmiwa-lab/05_231211_RePRSTrials/targetdata/Tongan.QC.FIXP.phased.annotated.vcf.gz


In [18]:
%%bash
cd ~/hmiwa1/storage/hmiwa-lab/06_231226_PRSadvance
mkdir -p analysis
ls

analysis
testdir


In [22]:
%%bash
pwd
cd ~/hmiwa1/storage/hmiwa-lab/06_231226_PRSadvance
pwd
cp ~/hmiwa1/storage/hmiwa-lab/05_231211_RePRSTrials/targetdata/Tongan.QC.FIXP.phased.annotated.vcf.gz* analysis/

/home/hmiwa/hmiwa/hmiwa-lab/d_20231023/06_231226_PRSadvance
/home/hmiwa/hmiwa1/storage/hmiwa-lab/06_231226_PRSadvance


In [26]:
%%bash
ls ~/hmiwa1/storage/hmiwa-lab/06_231226_PRSadvance/analysis/

Tongan.QC.FIXP.phased.annotated.vcf.gz
Tongan.QC.FIXP.phased.annotated.vcf.gz.csi


In [28]:
%%bash
#PRSを計算した時に使ったGWAS結果のチェック
ls ~/hmiwa1/storage/hmiwa-lab/05_231211_RePRSTrials/Basedata/BMI_Eastern_Asian.QC2.txt

/home/hmiwa/hmiwa1/storage/hmiwa-lab/05_231211_RePRSTrials/Basedata/BMI_Eastern_Asian.QC2.txt


In [29]:
%%bash
cd ~/hmiwa1/storage/hmiwa-lab/06_231226_PRSadvance
cp ~/hmiwa1/storage/hmiwa-lab/05_231211_RePRSTrials/Basedata/BMI_Eastern_Asian.QC2.txt analysis/

関数定義コードも用意しておこう。

In [32]:
%%bash
ls ~/hmiwa1/storage/hmiwa-lab/06_231226_PRSadvance/testdir/helper_functions_coal_sel.R

/home/hmiwa/hmiwa1/storage/hmiwa-lab/06_231226_PRSadvance/testdir/helper_functions_coal_sel.R


In [37]:
%%bash
cd ~/hmiwa1/storage/hmiwa-lab/06_231226_PRSadvance/
pwd
ls
cp testdir/helper_functions_coal_sel.R analysis/

/home/hmiwa/hmiwa1/storage/hmiwa-lab/06_231226_PRSadvance
analysis
testdir


RentPlusも必要ですね

In [48]:
%%bash
ls ~/hmiwa1/storage/hmiwa-lab/06_231226_PRSadvance/testdir/RentPlus.jar

/home/hmiwa/hmiwa1/storage/hmiwa-lab/06_231226_PRSadvance/testdir/RentPlus.jar


In [49]:
%%bash
cd ~/hmiwa1/storage/hmiwa-lab/06_231226_PRSadvance/
pwd
ls
cp testdir/RentPlus.jar analysis/

/home/hmiwa/hmiwa1/storage/hmiwa-lab/06_231226_PRSadvance
analysis
testdir


## 解析開始

### 解析データ形式等のチェック

In [43]:
%%bash
cd ~/hmiwa1/storage/hmiwa-lab/06_231226_PRSadvance/
ls

analysis
testdir


In [44]:
%%bash
cd ~/hmiwa1/storage/hmiwa-lab/06_231226_PRSadvance/testdir
ls

GBR_IDS.txt
height_lr_3ests.pdf
height_lr_3ests.tif
helper_functions_coal_sel.R
neale.height.best.snp.tsv
rent_1700_trees_1KG_height_UKB_neale_processed_072318.RData
rent_1700_trees_1KG_neale_height_071918.RData
RentPlus.jar
rentplus_test.RData
rentplus_trees_1700_height_1KG.R
Rplots.pdf
temp


In [46]:
%%bash
cd ~/hmiwa1/storage/hmiwa-lab/06_231226_PRSadvance/testdir
#id_file #sample name
head GBR_IDS.txt

HG00096
HG00097
HG00099
HG00100
HG00101
HG00102
HG00103
HG00105
HG00106
HG00107


In [51]:
%%bash
cd ~/hmiwa1/storage/hmiwa-lab/06_231226_PRSadvance/testdir
#height_fn #GWAS results
head -n 5 neale.height.best.snp.tsv

CHROM	POS	A1	A2	CHROM:POS	rsid	nCompleteSamples	AC	ytx	beta	se	tstat	pval	Block
1	958248	A	G	1:958248	rs2001744	336474	2.47091e+05	1.22927e+04	1.02295e-02	1.79298e-03	5.70527e+00	1.16258e-08	1
1	2142518	A	C	1:2142518	rs380190	336474	1.87183e+05	1.05732e+04	1.59487e-02	1.94239e-03	8.21087e+00	2.20363e-16	2
1	3769276	G	C	1:3769276	rs6668213	336474	4.63182e+05	1.88966e+04	-9.13069e-03	1.86504e-03	-4.89570e+00	9.79993e-07	3
1	4986470	G	T	1:4986470	rs2411956	336474	9.23723e+04	3.15752e+03	-9.00346e-03	2.53800e-03	-3.54746e+00	3.89012e-04	4


形式が違うので使われている場面をチェックします。

```R
#height_fnはここだけらしい
mark.dat <- fread(height_fn, head = TRUE)
```

```R
#mark.datを参照する行 #スクリプト構造無視
nmarks <- nrow(mark.dat)
snps.fail1 <- rep(0, dim(mark.dat)[1])
snps.miss <- rep(0, dim(mark.dat)[1])
for(i in 1:dim(mark.dat)[1]){
   snp <- mark.dat[i,6]
   pos.snp <- mark.dat[i,2]
   chr <- mark.dat[i,1]
mean(as.character(mark.dat$A2) == info.sites.char[,4]) #allele1 matches A2 in each case, polarization good.
mean(as.character(mark.dat$rsid) == info.sites.char[,1]) #check that the rsids match
beta <- mark.dat$beta
rsID <- mark.dat$rsid
```

neale.height.best.snp.tsvのheader
1. CHROM	
2. POS	
3. A1	
4. A2	
5. CHROM:POS	
6. rsid	
7. nCompleteSamples	
8. AC	
9. ytx	
10. beta	
11. se	
12. tstat	
13. pval	
14. Block